In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

In [39]:
# create data with pandas
import pandas as pd
data = {
    "DATE": ["2020", "2019", "2018", "2020", "2019", "2018"],
    "VALUE": [1, 3, 4, 5, 9, 6]
}
pd_df  = pd.DataFrame(data)

# make it a spark dataframe
spark_df = spark.createDataFrame(pd_df)
spark_df.show()

'''
Here is the shown input dataframe
+----+-----+
|DATE|VALUE|
+----+-----+
|2020|    1|
|2019|    3|
|2018|    4|
|2020|    5|
|2019|    9|
|2018|    6|
+----+-----+
'''

# perform window operations
from pyspark.sql.window import Window
from pyspark.sql.functions import max, sum

w = Window.partitionBy("DATE")
w_preceding = Window.partitionBy("DATE").orderBy('VALUE').rangeBetween(Window.unboundedPreceding, 0)

spark_df = spark_df.withColumn("MAX_WINDOW", max("VALUE").over(w))
spark_df = spark_df.withColumn("SUM_WINDOW", sum("VALUE").over(w))
spark_df = spark_df.withColumn("CUM_SUM_WINDOW", sum("VALUE").over(w_preceding))

'''
here is the final result dataframe
+----+-----+----------+----------+--------------+
|DATE|VALUE|MAX_WINDOW|SUM_WINDOW|CUM_SUM_WINDOW|
+----+-----+----------+----------+--------------+
|2020|    1|         5|         6|             1|
|2020|    5|         5|         6|             6|
|2019|    3|         9|        12|             3|
|2019|    9|         9|        12|            12|
|2018|    4|         6|        10|             4|
|2018|    6|         6|        10|            10|
+----+-----+----------+----------+--------------+
'''

spark_df.show()

+----+-----+
|DATE|VALUE|
+----+-----+
|2020|    1|
|2019|    3|
|2018|    4|
|2020|    5|
|2019|    9|
|2018|    6|
+----+-----+

+----+-----+----------+----------+--------------+
|DATE|VALUE|MAX_WINDOW|SUM_WINDOW|CUM_SUM_WINDOW|
+----+-----+----------+----------+--------------+
|2020|    1|         5|         6|             1|
|2020|    5|         5|         6|             6|
|2019|    3|         9|        12|             3|
|2019|    9|         9|        12|            12|
|2018|    4|         6|        10|             4|
|2018|    6|         6|        10|            10|
+----+-----+----------+----------+--------------+

